In [1]:
%matplotlib inline
import mxnet as mx
from mxnet import gluon, nd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from utils import GluonConfigs

In [2]:
tf = GluonConfigs.load_data()

In [3]:
tf = tf.set_index('transect_id')
tf = tf.dropna(thresh=33)
print(f"Transects included in dataset: {tf.shape[0]}; timesteps: {tf.shape[1]}")
tf.head()

Transects included in dataset: 37111; timesteps: 33


,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
transect_id,,,,,,,,,,,,,,,,,,,,,
BOX_051_151_15,677.32043,695.66370,713.29486,682.1743,688.23425,672.13007,700.43286,693.43460,699.38560,687.97950,...,700.6640,712.41740,708.45123,740.33240,680.42510,758.35150,754.49695,763.04297,743.29680,779.41570
BOX_051_151_18,768.93800,769.23883,762.44300,755.8622,761.79663,760.51710,763.30505,761.81360,769.22100,765.47060,...,763.1057,759.72600,766.23150,770.77130,798.26830,797.99615,803.90950,797.90780,798.18830,803.68256
BOX_051_151_21,711.41626,684.20540,695.49817,701.6695,705.29990,703.61470,707.69403,692.90360,704.68280,704.99945,...,825.8698,820.51720,820.68680,842.18097,850.45460,775.96700,883.04240,867.92426,877.37415,874.50244
BOX_051_151_30,795.81573,820.63700,799.72380,799.8664,823.56260,822.07300,823.56800,823.73300,824.58400,824.33360,...,823.7820,823.15674,823.65790,821.31860,818.85803,817.31220,822.92970,818.73350,822.18480,818.73480
BOX_051_151_32,242.70204,238.05159,229.93718,244.0620,247.68105,257.49900,302.69217,301.27722,301.12033,316.40414,...,301.1231,313.63553,313.09814,312.90347,306.18658,294.18326,297.23654,302.88650,323.85840,304.36823


In [4]:
tf = tf.sample(1000)

In [5]:
sites = pd.read_csv("/media/storage/data/shorelines/sites-gluonts-prepared.csv")
sites = sites.loc[sites['transect_id'].isin(tf.index)]

logs = np.log(sites['changerate_unc'].values)
sites['changerate_cat'] = pd.qcut(logs, q=10, labels=range(10))

In [6]:
def get_timestamp_first_obs(series: pd.Series) -> pd.Timestamp:
  yr = series.first_valid_index()
  return pd.Timestamp(f"01-01-{yr}", freq='1Y')

metadata = {'num_series': len(tf),
            'num_steps': len(tf.columns),
            'prediction_length': 7,
            'freq': "AS",
            'start': [pd.Timestamp("01-01-1984", freq='AS') for _ in range(len(tf))],
            'item_id': tf.index.values,
            'context_length': 26
            }

In [7]:
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset

train_ds = ListDataset(
    [
     {
         FieldName.TARGET: target,
         FieldName.START: start,
         FieldName.ITEM_ID: item_id,
         FieldName.FEAT_STATIC_CAT: [fsc1, fsc2],
         FieldName.FEAT_STATIC_REAL: [fsr]
      }
     
     for (target, start, item_id, fsc1, fsc2, fsr) in zip(tf.values[:, :-metadata['prediction_length']],
                                        metadata['start'], 
                                        metadata['item_id'],
                                        sites['coastline_idint'].values,
                                        sites['changerate_cat'].values,
                                        sites['changerate_unc'].values)
    ], freq=metadata['freq'])

test_ds = ListDataset(
    [
     {
         FieldName.TARGET: target,
         FieldName.START: start,
         FieldName.ITEM_ID: item_id,
         FieldName.FEAT_STATIC_CAT: [fsc1, fsc2],
         FieldName.FEAT_STATIC_REAL: [fsr]
      }
     
     for (target, start, item_id, fsc1, fsc2, fsr) in zip(tf.values,
                                        metadata['start'], 
                                        metadata['item_id'],
                                        sites['coastline_idint'].values,
                                        sites['changerate_cat'].values,
                                        sites['changerate_unc'].values)
    ], freq=metadata['freq'])

In [8]:
next(iter(train_ds))

{'target': array([787.0619 , 789.99536, 791.3418 , 784.9836 , 799.69147, 793.4877 ,
        788.6033 , 788.66046, 784.8622 , 780.5118 , 805.3251 , 790.579  ,
        809.4227 , 774.3561 , 817.7444 , 788.6603 , 809.219  , 809.20526,
        809.2215 , 805.1269 , 788.94464, 809.21765, 809.1172 , 812.491  ,
        821.6146 , 809.219  ], dtype=float32),
 'start': Timestamp('1984-01-01 00:00:00', freq='AS-JAN'),
 'item_id': 'BOX_074_041_17',
 'feat_static_cat': array([7427,    6], dtype=int32),
 'feat_static_real': array([0.2793744], dtype=float32),
 'source': SourceContext(source='list_data', row=0)}

In [ ]:
class MyNetwork(gluon.HybridBlock):
    def __init__(self, prediction_length, num_cells, **kwargs):
        super().__init__(**kwargs)
        self.prediction_length = prediction_length
        self.num_cells = num_cells

        with self.name_scope():
            # Set up a 3 layer neural network that directly predicts the target values
            self.nn = mx.gluon.nn.HybridSequential()
            self.nn.add(mx.gluon.nn.Dense(units=self.num_cells, activation='relu'))
            self.nn.add(mx.gluon.nn.Dense(units=self.num_cells, activation='relu'))
            self.nn.add(mx.gluon.nn.Dense(units=self.prediction_length, activation='softrelu'))


class MyTrainNetwork(MyNetwork):
    def hybrid_forward(self, F, past_target, future_target):
        prediction = self.nn(past_target)
        # calculate L1 loss with the future_target to learn the median
        return (prediction - future_target).abs().mean(axis=-1)


class MyPredNetwork(MyTrainNetwork):
    # The prediction network only receives past_target and returns predictions
    def hybrid_forward(self, F, past_target):
        prediction = self.nn(past_target)
        return prediction.expand_dims(axis=1)

